# 삼성 DS 과제 - DnCNN 모델 학습 파이프라인 (On-the-fly Augmentation Ver.)

이 노트북은 "On-the-fly Augmentation" 방식을 사용하여 Denoising, Deconvolution, End-to-End 모델을 학습시키는 파이프라인입니다.

**주요 특징:**
- **실시간 데이터 증강:** 대용량 데이터셋을 미리 생성할 필요 없이, 학습 중에 실시간으로 원본 이미지에 손상(Noise, Convolution)을 적용합니다. 이를 통해 I/O 병목 현상을 해결하고 저장 공간을 절약합니다.
- **통합 환경:** Denoising, Deconvolution, End-to-End 세 가지 실험을 노트북 셀 선택을 통해 간단하게 실행할 수 있습니다.
- **로컬 환경 호환:** `params.py`에 환경 자동 감지 기능이 추가되어, 이 노트북과 동일한 코드를 로컬 Windows 환경에서도 (CPU로) 실행할 수 있습니다.

**실행 순서:**
1. **1. 환경 설정 및 데이터 준비:** Drive 마운트, 경로 설정, 라이브러리 설치 및 원본 데이터셋(train, val)을 Colab 로컬 런타임으로 복사합니다. (최초 1회 필수 실행)
2. **2. 학습 실행:** 원하는 실험(Denoising, Deconvolution, End-to-End)에 해당하는 셀을 선택하여 실행합니다. 
   - 로그와 학습된 모델(`.ckpt`), 테스트 결과물은 모두 Google Drive의 프로젝트 폴더 내 `logs_...` 디렉토리에 저장됩니다.


## 1. 환경 설정 및 데이터 준비


In [ ]:
# @title Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# @title 프로젝트 경로 설정 및 라이브러리 설치
import os
import sys

# 💥 사용자 설정: 자신의 Google Drive 프로젝트 경로에 맞게 수정해주세요.
PROJECT_ROOT = "/content/drive/MyDrive/Data Scientist/Project/Week5/week5"
os.chdir(PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)

print(f"Current working directory: {os.getcwd()}")

# 필요한 라이브러리 설치
%pip install loguru tqdm --quiet


In [ ]:
# @title 🚀 [필수] 원본 데이터셋 로컬 런타임으로 복사
import shutil
from pathlib import Path
from tqdm.notebook import tqdm

DRIVE_DATASET_ROOT = Path(PROJECT_ROOT) / "dataset"
LOCAL_DATASET_ROOT = Path("/content/dataset")

# 복사할 폴더 목록 (원본 데이터만)
folders_to_copy = ["train", "val"]

print("Starting dataset copy from Google Drive to local runtime...")

if LOCAL_DATASET_ROOT.exists():
    print("Local dataset folder already exists. Skipping copy.")
else:
    LOCAL_DATASET_ROOT.mkdir(parents=True, exist_ok=True)
    
    total_files = 0
    for folder in folders_to_copy:
        source_folder = DRIVE_DATASET_ROOT / folder
        if source_folder.exists():
            # 전체 파일 수 계산
            total_files += len(list(source_folder.glob('**/*')))

    with tqdm(total=total_files, unit="file") as pbar:
        for folder in folders_to_copy:
            source_folder = DRIVE_DATASET_ROOT / folder
            target_folder = LOCAL_DATASET_ROOT / folder
            
            if not source_folder.exists():
                print(f"Warning: Source folder '{source_folder}' not found in Drive.")
                continue
                
            if not target_folder.exists():
                target_folder.mkdir(parents=True)
            
            pbar.set_description(f"Copying {folder}...")
            for file_path in source_folder.glob('**/*'):
                if file_path.is_file():
                    shutil.copy(file_path, target_folder / file_path.name)
                    pbar.update(1)

    print("\nDataset copy finished!")

# params.py가 로컬 경로를 바라보도록 환경 변수 설정
os.environ['DATA_ROOT'] = str(LOCAL_DATASET_ROOT)
print(f"DATA_ROOT is now set to: {os.environ.get('DATA_ROOT')}")


## 2. 학습 실행

아래 3개의 셀 중 원하는 실험을 선택하여 실행하세요. 로그와 결과물은 Drive에 저장됩니다.


In [ ]:
# @title 🧪 1. Denoising 모델 학습 (Noise Only)
!python ./code_denoising/train.py --run_dir_name "logs_denoising" --augmentation_mode "noise_only" --train_epoch 100 --train_batch 4 --valid_batch 8 --num_workers 4 --lr 1e-4


In [ ]:
# @title 🧪 2. Deconvolution 모델 학습 (Convolution Only)
!python ./code_denoising/train.py --run_dir_name "logs_deconvolution" --augmentation_mode "conv_only" --train_epoch 100 --train_batch 4 --valid_batch 8 --num_workers 4 --lr 1e-4


In [1]:
# @title 🧪 3. End-to-End 모델 학습 (Noise + Convolution)
!python ./code_denoising/train.py --run_dir_name "logs_end_to_end" --augmentation_mode "both" --train_epoch 100 --train_batch 4 --valid_batch 8 --num_workers 4 --lr 1e-4


^C


[INFO] Running on Windows. Forcing num_workers to 0 (was 4).
[INFO] Found 1 GPUs. Disabling DataParallel.
[DEBUG] train.py script started.
2025-08-19 16:42:09 [INFO] ####################################################################################################
2025-08-19 16:42:09 [INFO] Run dir: logs_end_to_end\00000_train
2025-08-19 16:42:09 [INFO] ####################################################################################################
2025-08-19 16:42:09 [INFO] General Config
2025-08-19 16:42:09 [INFO] train_dataset:['dataset/train']
2025-08-19 16:42:09 [INFO] valid_dataset:['dataset/val']
2025-08-19 16:42:09 [INFO] test_dataset:['dataset/val']
2025-08-19 16:42:09 [INFO] data_type:*.npy
2025-08-19 16:42:09 [INFO] log_lv:INFO
2025-08-19 16:42:09 [INFO] run_dir:logs_end_to_end
2025-08-19 16:42:09 [INFO] init_time:1755589329.5358129
2025-08-19 16:42:09 [INFO] augmentation_mode:both
2025-08-19 16:42:09 [INFO] noise_levels:[0.07, 0.132]
2025-08-19 16:42:09 [INFO] conv_di